### Preparação dos Dados para o Modelo

#### Tarefa

Exp


In [1]:
# LEITURA DOS DADOS E CARREGAMENTO DAS BIBLIOTECAS

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_pickle('../data/interim/ambev-final-dataset.pkl')

In [4]:
cols = ['peso_kpi', 'ating_mes', 'pts_mes','acum_mes', 'ating_acum', 'pts_acumulado',
'acum_acumulado', 'ating_fim_exer', 'pts_fim_exer', 'acum_fim_exer']

In [5]:
l = list(df.prazo.value_counts().index)
l = [str(i).replace(',','') for i in l]

In [6]:
(df.groupby(['id_funcionario']).mes.agg('max') - df.groupby(['id_funcionario']).mes.agg('min')).describe()

count    7858.000000
mean        6.814838
std         2.670806
min         0.000000
25%         6.000000
50%         8.000000
75%         9.000000
max         9.000000
Name: mes, dtype: float64

In [8]:
# REMOCAO COLUNAS SEM STATUS META
df = df[df.status_meta.notna()]

In [9]:
# REMOCAO DE COLUNAS COM UM ÚNICO VALOR
df = df[df.columns[df.nunique()>1]]

In [10]:
# COLUNAS PTS_*  E ATING* TEM A MESMA DISTRIBUIÇÃO
df.groupby(['ating_mes', 'pts_mes']).size()

ating_mes  pts_mes
100.0      100.0          2
0          0          29817
100        100        63839
80         60          7132
90         80          5496
dtype: int64

In [11]:
# REMOCAO DE COLUNAS PTS_* e ACUM_* 
# elas são altamente correlacionadas com o nosso target
df = df.drop(['pts_mes', 'pts_acumulado', 'pts_fim_exer'], axis=1)
df = df.drop(['acum_mes', 'acum_acumulado', 'acum_fim_exer'], axis=1)

In [12]:
# M
# df = df.rename(columns = {'ating_mes':'target'})

In [13]:
# Calcular target como média do ating_mes por meta
df['ating_mes'] = df.ating_mes.astype(np.number).fillna(0)

In [14]:
df['target'] = df.groupby(['id_funcionario', 'nome_kpi']).ating_mes.transform('mean') / 100

In [15]:
df = df.groupby(['id_funcionario', 'nome_kpi']).agg('first')

In [16]:
df.target.describe()

count    16986.000000
mean         0.690637
std          0.334422
min          0.000000
25%          0.487500
50%          0.800000
75%          1.000000
max          1.000000
Name: target, dtype: float64

In [17]:
df[df.id_funcionario == 7052].ating_mes.describe()

AttributeError: 'DataFrame' object has no attribute 'id_funcionario'

In [ ]:
200 % 100 == 0

In [ ]:
sns.distplot(df.target)

In [ ]:
# REMOCAO DE COLUNAS DE IDENTIFICAÇÃO - ID_FUNCIONARIO / ID_GESTOR
# df = df.drop(['id_funcionario', 'id_gestor'], axis=1)

In [ ]:
# Pegar as informações dos funcionários do primeiro mês deles
df_modelo = df.groupby(['id_funcionario']).agg('first').reset_index()

In [ ]:
# REMOCAO DE COLUNAS DE IDENTIFICAÇÃO - ID_FUNCIONARIO / ID_GESTOR
df_modelo = df_modelo.drop(['id_funcionario', 'id_gestor'], axis=1)

In [ ]:
df_modelo

In [ ]:
df['peso_mult1'] = df['pts_mes'] * df['peso_kpi'] / 100
df['peso_mult2'] = df['pts_acumulado'] * df['peso_kpi'] / 100
df['peso_mult3'] = df['pts_fim_exer'] * df['peso_kpi'] / 100